In [1]:
import json
import numpy as np
from pathlib import Path
from numba import jit
from datetime import datetime, timedelta
from collections import Counter

from overprot.libs import lib_alignment
from overprot.libs import lib_pymol
from overprot.libs.lib_structure import Structure
from overprot.libs.lib import Timing

 PyMOL not running, entering library mode (experimental)


In [13]:
DATA = Path('/home/adam/Workspace/Python/OverProt/data/tree/structs')
pdbs = ['1tqn', '2nnj', '1cbs']
for pdb in pdbs:
    struct: Structure = lib_pymol.read_cif(DATA / f'{pdb}.cif')
    struct = struct.get_alpha_trace()
    struct.save_cif(DATA / f'{pdb}.alphas.cif')
with Timing():
    A = '2nnj'
    B = '1tqn'
    # lib_pymol.cealign(DATA/f'{A}.alphas.cif', DATA/f'{B}.alphas.cif')
    d = lib_alignment.dist(DATA/f'{A}.alphas.cif', DATA/f'{B}.alphas.cif')
    print(d)

130.80022813007236
Timing: 0:00:00.155455


In [37]:
import tree_stuff as ts
n = 885
domains = np.array(ts.get_domains(n))
families = np.array([famdom.split('/')[1] for famdom in (ts.DATA / f'choice_{n}.txt').read_text().split()])
fam_counts = Counter(families)

In [42]:
def distance_badness(distance_matrix: Path, domains, families, verbose=False):
    n = len(domains)
    distances = np.loadtxt(distance_matrix)
    distances_wo_self = distances.copy()
    distances_wo_self[range(n), range(n)] = np.inf
    # distances_wo_self[:5,:5]
    nns = [np.argmin(distances_wo_self[i]) for i in range(n)]
    good = []
    bad = []
    bad_T = []  # subset of bad, put in wrong Topology
    bad_A = []  # subset of bad_T, put in wrong Architecture
    bad_C = []  # subset of bad_A, put in wrong Class
    for i, nn in enumerate(nns):
        # print(domains[i], families[i], domains[nn], families[nn])
        if families[i] == families[nn]:
            good.append(i)
        else:
            bad.append(i)
            if families[i].split('.')[:3] != families[nn].split('.')[:3]:
                bad_T.append(i)
    if verbose:
        for i in bad:
            nn = nns[i]
            print(domains[i], domains[nn], f'{families[i]}   ({fam_counts[families[i]]})', f'{families[nn]}   ({fam_counts[families[nn]]})', sep='\t')
    return len(bad)/n, len(bad_T)/n, len(bad_A)/n, len(bad_C)/n, good, bad
print('lengthmax', distance_badness(ts.DATA / f'distance_{n}x{n}-lengthmax.csv', domains, families)[0])
print(3, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd3.csv', domains, families)[0])
print(5, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd5.csv', domains, families)[0])
print(7, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd7.csv', domains, families)[0])
print(8, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd8.csv', domains, families)[0])
print(10, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd10.csv', domains, families)[0])
print(12, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd12.csv', domains, families)[0])
print(15, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd15.csv', domains, families)[0])
print(20, distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd20.csv', domains, families)[0])
print('lengthmin', distance_badness(ts.DATA / f'distance_{n}x{n}-lengthmin.csv', domains, families)[0])

lengthmax 0.9954802259887006
3 0.33220338983050846
5 0.18983050847457628
7 0.1672316384180791
8 0.16610169491525423
10 0.17175141242937852
12 0.1864406779661017
15 0.2214689265536723
20 0.25649717514124293
lengthmin 0.8531073446327684


In [43]:
# distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd7.csv', domains, families)
distance_badness(ts.DATA / f'distance_{n}x{n}-maxrmsd7.csv', domains, families, verbose=True)

2o3fA00	3rbdA03	1.10.10.10   (15)	1.10.150.20   (12)
3fxrA01	3kxbA00	1.10.10.10   (15)	1.10.20.10   (2)
4q5sF02	1jtxA01	1.10.10.10   (15)	1.10.10.60   (7)
1d0kA02	3r2sA00	1.10.530.10   (11)	1.20.1260.10   (4)
4ndbA00	6g5aA00	1.10.238.10   (8)	1.10.490.10   (12)
1hdpA00	2m8eA00	1.10.10.60   (7)	1.10.10.10   (15)
1jtxA01	4q5sF02	1.10.10.60   (7)	1.10.10.10   (15)
2m34A00	2m8eA00	1.10.10.60   (7)	1.10.10.10   (15)
2r0qC02	4q5sF02	1.10.10.60   (7)	1.10.10.10   (15)
4omvA02	1tf0A01	1.20.1050.10   (5)	1.10.246.10   (1)
1zzhA01	3fxrA01	1.10.760.10   (4)	1.10.10.10   (15)
1tf0A01	2o3fA00	1.10.246.10   (1)	1.10.10.10   (15)
1egjA00	1pfsA00	2.60.40.10   (69)	2.40.50.140   (10)
1wunT01	1pfsA00	2.60.40.10   (69)	2.40.50.140   (10)
2fh6A02	3e0eA00	2.60.40.10   (69)	2.40.50.140   (10)
4ogzA01	3zoaA03	2.60.40.10   (69)	2.60.40.1180   (8)
2gtbA01	2mq0A00	2.40.10.10   (26)	2.60.40.10   (69)
2z6bD01	3zoaA03	2.40.10.10   (26)	2.60.40.1180   (8)
3proA01	1ckaA00	2.40.10.10   (26)	2.30.30.40   (5)
3sueA02	5

(0.1672316384180791,
 0.1401129943502825,
 0.0,
 0.0,
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  53,
  54,
  55,
  56,
  57,
  59,
  60,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  143,
  146,
  147,
  148,
  149,
  150,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  160,
  1

In [3]:
np.argmin(distances[0])
# np.min(distances[0])
nans = np.where(np.isnan(distances[0]))
nans, domains[nans], families[nans]

((array([], dtype=int64),), array([], dtype='<U7'), array([], dtype='<U13'))

In [67]:
rotations = np.load(ts.DATA / 'rotations_20x20-op-lin10.npy')
translations = np.load(ts.DATA / 'translations_20x20-op-lin10.npy')
rotations.shape, translations.shape, rotations.max(), rotations.min(), translations.max(), translations.min()

((20, 20, 3, 3),
 (20, 20, 1, 3),
 1.0000000000000004,
 -0.9999609197964008,
 21.194041055790912,
 -23.839824719511864)

In [14]:
def seconds(s): return timedelta(seconds=s)

In [18]:
print(seconds(25) / 200**2 * 6565**2)
print(seconds(363) / 1000**2 * 6565**2)

7:28:57.015625
4:20:45.018675


In [59]:
print(seconds(25) /100**2 *885**2)
1378/7280*100, 3170/7280*100
885-388
print(timedelta(minutes=12) /885**2 *6565**2)
# distance matrix 200x200 (shapedist-maxrmsd7): 51 s
# distance matrix 200x200 (shapedist-maxrmsd7): 47 s (precomputed shapes)
# distance matrix 200x200 (shapedist-maxrmsd7): 42 s (precomputed shapes, omitting center point of the shape)

0:32:38.062500
11:00:08.187775


In [61]:
print(timedelta(minutes=12, seconds=25) /200**2 *885**2)

4:03:07.565625


In [28]:
A = np.arange(10)
B = A[np.newaxis, :]
B[0,5]=100
print(A)
print(B)

[  0   1   2   3   4 100   6   7   8   9]
[[  0   1   2   3   4 100   6   7   8   9]]
